In [1]:
import vision as vis
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos
import math

COMPUTE = 0
TRAVEL = 1
LOCAL_NAV = 2
FINISH = 3

In [2]:
get_prox = False
if get_prox == False:
    state = COMPUTE

if get_prox == True:
    state = LOCAL_NAV

In [ ]:
# Main -> state machine

obstacle = False
path_calculated = False

cap = cv.VideoCapture(0)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while(1):
    time.sleep(1)
    captured, img = cap.read()
    if captured:
        cropped, cropped_img = vis.crop_map(img)
        # print(cropped_img.shape)        # uncomment to debug
        # cv.imshow("Original", img)        # uncomment to debug
        cv.imshow("Cropped", cropped_img)
        if cropped:
            im3 = np.zeros(cropped_img.shape)
            found, obstacles = vis.extract_obstacles(cropped_img)
            if found:
                ex_obstacles = vis.expand_obstacles(obstacles)
                cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                cv.imshow("expanded", im3)
            found_rob, robot_pos = pos.get_robot_position(cropped_img)
            if found_rob:
                vis.annotate_robot(robot_pos, im3)
            found_goal, goal_pos = pos.get_goal_position(cropped_img)
            if found_goal:
                vis.annotate_goal(goal_pos, im3)
            found_arch, arch_pos = pos.get_arch_positions(cropped_img)
            if found_arch and found_rob:
                if math.dist(arch_pos[0],robot_pos[0])>math.dist(arch_pos[1],robot_pos[0]):
                    arch_pos = np.flip(arch_pos,0)
                vis.annotate_arch(arch_pos, im3)
            if (found and found_rob and found_goal and found_arch):
                if path_calculated == False:
                    vertices = vis.convert_vertice(ex_obstacles)
                    pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(arch_pos[0]))
                    path_arch =\
                    short.names_to_subpaths(pathname,vertices,tuple(robot_pos[0]), tuple(arch_pos[0]))[0]
                    vis.annotate_path(path_arch,im3)
                    
                    pathname2 = short.find_shortest_path(vertices, tuple(arch_pos[1]), tuple(goal_pos))
                    path_goal =\
                    short.names_to_subpaths(pathname2,vertices,tuple(arch_pos[1]), tuple(goal_pos))[0]
                    print("namespath", pathname2)
                    print("path",path_goal)
                    vis.annotate_path(path_goal,im3)
                    state = TRAVEL
                    break
            cv.imshow("environment", im3)
            cv.waitKey(0)
    else:
        print("There was a problem in the capture")
        break
cv.imshow("environment",im3)
cv.waitKey(0)
cap.release()
print("finished!")
# cv.destroyAllWindows()
    

"""
# robot run    
elif short.find_shortest_path() == True:
    state = 1

obstacle = check_obstacle(prox_horizontal)
    
        # global navigation
        if  obstacle == False:
            motion_control

        # local navigation
        elif obstacle == True:
"""
            
            
        
    

C:\Users\sylpe\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:804: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


no circle found
None
no circle found
no circle found
no circle found
[[112  58  17]
 [ 72  62  18]]

coordinates of circle is: [[112  58  17]
 [ 72  62  18]] 
number of circles detected is: 2
4 40
Start (72, 62)
Goal (404, 100)
Vertices [[(495, 259), (555, 363), (433, 432), (373, 330)], [(244, 184), (371, 277), (282, 370)], [(142, 114), (195, 227), (67, 286), (14, 175)], [(410, -11), (478, -6), (467, 211), (403, 207)], [(288, -17), (382, 104), (272, 176), (256, 8)]]
list neighbours [[(5, 210.87437018281761), (8, 87.20091742636656), (11, 127.01574705523721), (16, 229.99347816840373), (18, 230.20860105565126), (19, 191.7602670002313)], [(2, 120.06664815842908), (4, 141.15594213493105), (5, 261.9656466027559), (6, 125.29964086141668), (7, 240.18742681497713), (13, 265.5447231635379), (14, 55.569775957799216), (15, 105.67875850898325), (18, 237.94537188186703)], [(1, 120.06664815842908), (3, 140.1606221447379), (13, 376.9482723133242)], [(2, 140.1606221447379), (4, 118.33849753989612), (7,

In [ ]:
OBSTACLE_THRESHOLD 10

def check_obstacle(prox_horizontal):
    prox_values = [prox_horizontal[0], prox_horizontal[4]]
    if prox_values[0] > OBSTACLE_THRESHOLD or prox_values[1] > OBSTACLE_THRESHOLD:
        obstacle = True
        
    else:
        obstacle = False
        
        
    